# Download Financials and Price data from Yahoo Finance

## Summary of Workflow:
+ 1. Import Libraries: Import yfinance, pandas, and other required libraries.
+ 2. Define Tickers and Date Range: Set up the stock tickers and date range for querying historical data.
+ 3. Fetch and Save Stock Prices: Use yfinance to fetch historical stock prices and save them to a CSV.
+ 4. Fetch and Save Income Statements: Retrieve income statements and save to CSV.
+ 5. Fetch and Save Balance Sheets: Fetch balance sheets and save to CSV.
+ 6. Fetch and Save Cash Flow Statements: Fetch cash flow statements and save to CSV.
+ 7. Merge Financial Data: Merge all financial statements into a single DataFrame and save the final result.

# Step 1: Import Necessary Libraries


In [35]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
from datetime import datetime


+ yfinance: Used to fetch stock price data, income statements, balance sheets, and cash flow statements.
+ pandas: Used for data manipulation and analysis.
+ datetime: Helps us handle date formatting for querying stock data.

# Step 2: Define the List of Stock Tickers and Date Range


In [51]:
# Define the stock ticker symbols and the date range
tickers = ['NVDA']

start_date = '2020-01-01'
end_date = '2024-12-31'

+ We define a list of S&P 100 stock tickers for major companies and specify a date range from January 1, 2020, to December 31, 2022. This range will be used to query historical stock prices.

# Step 3: Fetch Historical Stock Prices


In [52]:
# Initialize an empty DataFrame to store the stock price data
all_stock_prices = pd.DataFrame()

max_retries = 5  # Maximum number of retries for each request
retry_delay = 5  # Delay between retries in seconds

def fetch_stock_data(ticker):
    """
    Fetches stock data for a given ticker with retries.
    """
    for attempt in range(max_retries):
        try:
            # Fetch historical stock prices using yfinance
            stock_data = yf.Ticker(ticker)
            history = stock_data.history(start=start_date, end=end_date)

            # Check if the DataFrame is empty (indicating no data available, possibly delisted)
            if history.empty:
                print(f"No data found for {ticker}, possibly delisted or inactive.")
                return None

            # Create a DataFrame with the 'Close' prices and necessary columns
            stock_prices_df = history[['Close']].copy()
            stock_prices_df['Ticker'] = ticker
            stock_prices_df['Date'] = stock_prices_df.index
            stock_prices_df.rename(columns={'Close': 'Price'}, inplace=True)

            return stock_prices_df

        except Exception as e:
            print(f"Attempt {attempt+1} failed for {ticker}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # Wait before retrying
            else:
                print(f"Max retries exceeded for {ticker}. Skipping.")
                return None

# Iterate over each ticker symbol to fetch historical stock prices
for ticker in tickers:
    stock_prices_df = fetch_stock_data(ticker)
    if stock_prices_df is not None:
        # Concatenate the data for this ticker with the master DataFrame
        all_stock_prices = pd.concat([all_stock_prices, stock_prices_df])



+ We initialize an empty DataFrame to store stock price data for all companies. For each ticker, we use yfinance to get the historical stock prices (using the .history() method) and extract the closing price. The stock prices for each company are stored in the master DataFrame.

# Step 4: Clean and Save the Stock Prices Data


In [53]:
# Reset the index of the combined DataFrame
all_stock_prices.reset_index(drop=True, inplace=True)

# Reorder the DataFrame columns
all_stock_prices = all_stock_prices[['Ticker', 'Date', 'Price']]

# Save the stock prices to a CSV file
all_stock_prices.to_csv('output-csv-yahoo/all_stock_prices.csv', index=False)

print("Stock prices saved successfully.")


Stock prices saved successfully.


# Step 5: Fetch Income Statements for Each Ticker


In [54]:
# Initialize an empty DataFrame to store all income statements
all_income_statements = pd.DataFrame()

for ticker in tickers:
    try:
        stock_data = yf.Ticker(ticker)
        income_statement = stock_data.financials.T  # Transpose for better structure
        income_statement['Ticker'] = ticker
        income_statement.reset_index(inplace=True)
        income_statement.rename(columns={'index': 'Date'}, inplace=True)
        print(income_statement.Date)
        all_income_statements = pd.concat([all_income_statements, income_statement])
    except Exception as e:
        print(f"Failed to fetch income statement for {ticker}: {e}")


0   2025-01-31
1   2024-01-31
2   2023-01-31
3   2022-01-31
Name: Date, dtype: datetime64[ns]


# Step 6: Clean and Save the Income Statements Data


In [55]:
# Reorder the columns to have 'Ticker' and 'Date' first
columns = ['Ticker', 'Date'] + [col for col in all_income_statements.columns if col not in ['Ticker', 'Date']]
all_income_statements = all_income_statements[columns]

# Save the income statements to a CSV file
all_income_statements.to_csv('output-csv-yahoo/all_income_statements.csv', index=False)


# Step 7: Fetch and Save Balance Sheets


In [56]:
# Initialize an empty DataFrame to store all balance sheets
all_balance_sheets = pd.DataFrame()

for ticker in tickers:
    try:
        stock_data = yf.Ticker(ticker)
        balance_sheet = stock_data.balance_sheet.T  # Transpose for better structure
        balance_sheet['Ticker'] = ticker
        balance_sheet.reset_index(inplace=True)
        balance_sheet.rename(columns={'index': 'Date'}, inplace=True)
        all_balance_sheets = pd.concat([all_balance_sheets, balance_sheet])
    except Exception as e:
        print(f"Failed to fetch balance sheet for {ticker}: {e}")

# Reorder and save the balance sheet data
columns = ['Ticker', 'Date'] + [col for col in all_balance_sheets.columns if col not in ['Ticker', 'Date']]
all_balance_sheets = all_balance_sheets[columns]

# Save the balance sheet data to a CSV file
all_balance_sheets.to_csv('output-csv-yahoo/all_balance_sheets.csv', index=False)


# Step 8: Fetch and Save Cash Flow Statements


In [57]:
# Initialize an empty DataFrame to store all cash flow statements
all_cashflow = pd.DataFrame()

for ticker in tickers:
    try:
        stock_data = yf.Ticker(ticker)
        cashflow = stock_data.cashflow.T  # Transpose for better structure
        cashflow['Ticker'] = ticker
        cashflow.reset_index(inplace=True)
        cashflow.rename(columns={'index': 'Date'}, inplace=True)
        all_cashflow = pd.concat([all_cashflow, cashflow])
    except Exception as e:
        print(f"Failed to fetch cash flow statement for {ticker}: {e}")

# Reorder and save the cash flow data
columns = ['Ticker', 'Date'] + [col for col in all_cashflow.columns if col not in ['Ticker', 'Date']]
all_cashflow = all_cashflow[columns]

# Save the cash flow statements to a CSV file
all_cashflow.to_csv('output-csv-yahoo/all_cashflow.csv', index=False)


# Step 9: Merge All Financial Statements


In [58]:
# Merge all financial statements (balance sheet, income statement, cash flow)
fs_merged1 = pd.merge(all_balance_sheets, all_income_statements, on=['Ticker', 'Date'], how='inner')
fs_merged = pd.merge(fs_merged1, all_cashflow, on=['Ticker', 'Date'], how='inner')

# Save the merged financial statements to a CSV file
fs_merged.to_csv('output-csv-yahoo/all_financial_statements.csv', index=False)
